### Imported the necessary libraries

In [15]:
import bs4
import requests 
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

### Defined the download function. All the images are stored in the 'Images generated' folder

In [16]:
def download_image(url, folder_name, num1):
    
    # write image to file
    reponse = requests.get(url)
    if reponse.status_code == 200:
        with open(os.path.join(folder_name, str(num1) + ".jpg"), 'wb') as file:
            file.write(reponse.content)
    
# create a directory to save images
folder_name = 'Images generated_horses' # name whatever you want
if not os.path.isdir(folder_name):
    os.makedirs(folder_name)

### Opened a chrome window

In [17]:
chromeDriverPath = r'C:\Users\birud\OneDrive - iitgn.ac.in\github\Web-Scraper\Drivers\chromedriver.exe'

# Create a ChromeOptions object instead of using the deprecated DesiredCapabilities
chrome_options = Options()

# Add any additional options you need, for example:
# chrome_options.add_argument("--headless")

# Use ChromeService instead of the deprecated Service
service = ChromeService(chromeDriverPath)

# Pass the chrome_options and service to the webdriver.Chrome constructor
driver = webdriver.Chrome(service=service, options=chrome_options)


### Enter the URL below and scroll the newly opened window to its bottom most point

In [18]:
inp = input("Enter url: ", )
search_url = inp
driver.get(search_url)

Enter url: https://www.google.com/search?q=horse+images&source=lnms&tbm=isch


### Enter the number of images to be generated

In [19]:
x = int(input("Enter the number of images required : ", ))

Enter the number of images required : 300


### Below is the code, run it.

In [21]:
a = input("Press any key and hit enter to start : ")
# Scrolling all the way up
driver.execute_script("window.scrollTo(0, 0);")
scroll_distance = 60 # You can change to improve performance

page_html = driver.page_source
pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')
containers = pageSoup.findAll('div', {'class':"isv-r PNCib ViTmJb BUooTd"}) # Choose the class based on the search_url

len_containers = len(containers)
print("Found %s image containers"%(len_containers))

# selector for first image in search_url : #islrg > div.islrc > div:nth-child(2)
# Xpath for first image in search_url : //*[@id="islrg"]/div[1]/div[1]

# Xpaths that should be ignored :
# 1) //*[@id="islrg"]/div[1]/div[25]
# 2) //*[@id="islrg"]/div[1]/div[50]

count_images_generated = 0
missed_list = []
timeout_seconds = 10
timeout = 10

# Loop 1
for i in range(1, x + 1):

    if count_images_generated == x:
        break
        
    if (len(missed_list) == 20):
        break
    xPath = '//*[@id="islrg"]/div[1]/div[%s]' % (i)

    try:
        # Grabbing the URL of the small preview image
        previewImageXPath = '//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img' % (i)
        
        # Adding a timeout for finding the preview image element
        previewImageElement = WebDriverWait(driver, 0.05).until(
            EC.presence_of_element_located((By.XPATH, previewImageXPath))
        )
        
        previewImageURL = previewImageElement.get_attribute("src")
        # clicking on the image container
        driver.find_element('xpath', xPath).click()

    except Exception as e:
        print(f"Xpath has changed, wait untill we collect the missed tag: {i}")
        driver.execute_script(f"window.scrollTo(0, {i*scroll_distance});")
        time.sleep(0.5)
        missed_list.append(i)
        continue

    # Starting a while True loop to wait until we see that the URL inside the large image view is different from the preview one
    timeStarted = time.time()
    while True:
        try:
            imageElement = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'))
            )
            imageURL = imageElement.get_attribute('src')

            if imageURL != previewImageURL:
                break
            else:
                # making a timeout if the full res image can't be loaded
                currentTime = time.time()

                if currentTime - timeStarted > timeout_seconds:
                    print("Timeout! Exiting the loop.")
                    break

        except Exception as e:
            print(f"Error: {e}. Couldn't retrieve full res image URL. Continuing to the next one.")
            break  # Continue to the next iteration if an error occurs

    # Try to download the image
    try:
        download_image(imageURL, folder_name, count_images_generated)
        print("Downloaded element %s. URL: %s" % (i, imageURL))
        count_images_generated += 1

    except Exception as e:
        print(f"Error: {e}. Couldn't download the image {i}, continuing downloading the next one.")

    time.sleep(1)  # Add a small delay to avoid overloading the server

# Loop 2
time.sleep(3)

for i in missed_list:
    if count_images_generated == x:
        break

    missed_count = 0
    for j in range(1, x + 1):
        k = i*j
        if count_images_generated == x:
            break
            
        if missed_count >= 5: # You can change to improve performance
#             driver.refresh()
            driver.execute_script(f"window.scrollTo(0, {k*scroll_distance});")
            time.sleep(2)

            page_html = driver.page_source
            pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')
            containers = pageSoup.findAll('div', {'class':"isv-r PNCib ViTmJb BUooTd"}) # Choose the class based on the search_url
            break

        xPath = '//*[@id="islrg"]/div[1]/div[%s]/div[%s]' % (i, j)

        try:
#             driver.execute_script(f"window.scrollTo(0, {(i+j)*scroll_distance});")
            previewImageXPath = '//*[@id="islrg"]/div[1]/div[%s]/div[%s]/a[1]/div[1]/img' % (i, j)
            # Adding a timeout for finding the preview image element
            previewImageElement = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, previewImageXPath))
            )
            
            previewImageURL = previewImageElement.get_attribute("src")

            # clicking on the image container
            driver.find_element('xpath', xPath).click()

        except Exception as e:
#             print(f"Error: {e}. Couldn't process image [{i}, {j}]. Continuing to the next one.")
            print(f"Couldn't find element [{i}, {j}]. Checking if Xpath has changed")
#             driver.execute_script(f"window.scrollTo(0, {k*scroll_distance});")
            time.sleep(1)
            missed_count += 1
            continue

        # Starting a while True loop to wait until we see that the URL inside the large image view is different from the preview one
        timeStarted = time.time()
        while True:
            try:
                imageElement = WebDriverWait(driver, timeout).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'))
                )
                imageURL = imageElement.get_attribute('src')

                if imageURL != previewImageURL:
                    break
                else:
                    # making a timeout if the full res image can't be loaded
                    currentTime = time.time()

                    if currentTime - timeStarted > timeout_seconds:
                        print("Timeout! Exiting the loop.")
                        break

            except Exception as e:
                print(f"Error: {e}. Couldn't retrieve full res image URL. Continuing to the next one.")
                break  # Continue to the next iteration if an error occurs

        # Try to download the image
        try:
            download_image(imageURL, folder_name, count_images_generated)
            print("Downloaded element [%s, %s]. URL: %s" % (i, j, imageURL))
            count_images_generated += 1
            missed_count = 0

        except Exception as e:
            print(f"Error: {e}. Couldn't download the image [{i}, {j}], continuing downloading the next one.")

        time.sleep(1)  # Add a small delay to avoid overloading the server

if x == count_images_generated:
    print("Target Reached")
else:
    print("There is some issue, scroll down the page and run code again")


Press any key and hit enter to start : a
Found 100 image containers
Downloaded element 1. URL: https://i.pinimg.com/originals/e9/0a/83/e90a83fee831ace29e796e41308b7b0b.jpg
Downloaded element 2. URL: https://img.freepik.com/free-photo/running-stallion-meadow-capturing-beauty-nature-generated-by-artificial-intelligence_25030-62873.jpg
Downloaded element 3. URL: https://i.pinimg.com/1200x/3b/6c/aa/3b6caa1e7f09d484cd9539236e4a34e8.jpg
Downloaded element 4. URL: https://w0.peakpx.com/wallpaper/256/27/HD-wallpaper-artistic-white-horses-with-sunset-background-horse.jpg
Downloaded element 5. URL: https://wallpapers-clan.com/wp-content/uploads/2023/03/horse-aesthetic-wallpaper.jpg
Downloaded element 6. URL: https://as2.ftcdn.net/v2/jpg/05/65/77/05/1000_F_565770562_YDaj9SiTfQEf8axvOpHDw4nCZqfqO6dr.jpg
Downloaded element 7. URL: https://redandwinedecor.in/wp-content/uploads/2023/04/207.jpg
Downloaded element 8. URL: https://w0.peakpx.com/wallpaper/106/1016/HD-wallpaper-horse-animal-race-running-t

Couldn't find element [50, 4]. Checking if Xpath has changed
Couldn't find element [50, 5]. Checking if Xpath has changed
Couldn't find element [51, 1]. Checking if Xpath has changed
Downloaded element [51, 2]. URL: https://m.media-amazon.com/images/W/MEDIAX_792452-T2/images/I/71IPZoE04ML._AC_UF1000,1000_QL80_.jpg
Downloaded element [51, 3]. URL: https://funmauj.b-cdn.net/test/385677.jpg
Downloaded element [51, 4]. URL: https://is1-ssl.mzstatic.com/image/thumb/Purple126/v4/02/75/6b/02756b70-0936-7957-ff9b-00f91a47ace1/AppIcon-0-0-1x_U007emarketing-0-7-0-85-220.png/512x512bb.jpg
Downloaded element [51, 5]. URL: https://png.pngtree.com/background/20230611/original/pngtree-black-horse-beautiful-portrait-wallpaper-hd-picture-image_3133861.jpg
Downloaded element [51, 6]. URL: https://backiee.com/static/wpdb/wallpapers/560x315/149750.jpg
Downloaded element [51, 7]. URL: https://w0.peakpx.com/wallpaper/852/381/HD-wallpaper-fire-horse-horse-ultimate-thumbnail.jpg
Downloaded element [51, 8]. UR

Downloaded element [52, 9]. URL: https://www.enjpg.com/img/2020/horse-78-e1664477792323.jpg
Downloaded element [52, 10]. URL: https://1474899231.rsc.cdn77.org/api/cc-wallpaper-image/200/420/horse-water-sky-x600smp
Downloaded element [52, 11]. URL: https://priceo.in/wp-content/uploads/2022/09/Horse-wallpaper-for-iphone-14-16.jpeg
Downloaded element [52, 12]. URL: https://wallpapercg.com/download/wild-horses-4000x3000-5694.jpg
Downloaded element [52, 13]. URL: https://images3.alphacoders.com/113/1133819.jpg
Error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)). Couldn't download the image [52, 14], continuing downloading the next one.
Downloaded element [52, 15]. URL: https://www.ixpap.com/images/2022/01/Running-Horse-Wallpaper.jpg
Downloaded element [52, 16]. URL: https://rukminim2.flixcart.com/image/850/1000/l071d3k0/poster/4/z/q/medium-running-horse-in-water-with-bird-paintings-on-fine-art-origin

Downloaded element [52, 73]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQOfH254Dsre0qm4kc0etbgakSm9TxTcDfFAQ&usqp=CAU
Downloaded element [52, 74]. URL: https://static.uwalls.com/products/96000/96645/u98192pir5m_900.webp
Downloaded element [52, 75]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQv8babw2iBmqN46hbM0snR0qWtls5DPArZ4Q&usqp=CAU
Downloaded element [52, 76]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRKWafPV7-YcXoD6R5Bb4EP6iEeuJEji5VtbQ&usqp=CAU
Downloaded element [52, 77]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS1ER-qCtd0d_EjaCr_isUJAVwRev_W6F2BXQ&usqp=CAU
Downloaded element [52, 78]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRXy_rGQ_uTwsW_wPlu-feupq-0fRa_idjwNw&usqp=CAU
Downloaded element [52, 79]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSRCuLJYKH99yM-XWqqFSs3yuLRcTj10g-jRQ&usqp=CAU
Downloaded element [52, 80]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRO86GO

Downloaded element [53, 25]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX9mfW-aWNYsV5S66sXLdGRAAZAELKPpKdA&usqp=CAU
Downloaded element [53, 26]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTCdqBnkoZfjzpiK2DL96GL8-iFxL2w168Few&usqp=CAU
Downloaded element [53, 27]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQeV-LM3rEbTpOLpg4Tm9ijwQNOf7DJ_LTVNg&usqp=CAU
Downloaded element [53, 28]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTlg422BUyOcmOE31JXKhzB7x8Zfko-Oc2wrg&usqp=CAU
Downloaded element [53, 29]. URL: https://www.whatspaper.com/wp-content/uploads/2023/01/hd-horse-wallpaper-whatspaper-2.webp
Downloaded element [53, 30]. URL: https://w0.peakpx.com/wallpaper/665/625/HD-wallpaper-white-horse-animal-horses-running.jpg
Downloaded element [53, 31]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQd83sGNy_B27cpWouprnWaDESINLff6ns-pQ&usqp=CAU
Downloaded element [53, 32]. URL: https://encrypted-tbn0.gstatic.com/images?q=tb

Downloaded element [53, 88]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ7YxGRhsMZWCKu1fR_uEFoHUKkvYVgZC3ZuA&usqp=CAU
Downloaded element [53, 89]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQbqgniL6MJ51mpeMvHV4sYNR9GxUvXMrJCfA&usqp=CAU
Downloaded element [53, 90]. URL: https://img.freepik.com/free-photo/beautiful-unicorn-nature-composition_23-2150169277.jpg?size=626&ext=jpg&ga=GA1.1.1412446893.1704499200&semt=ais
Downloaded element [53, 91]. URL: https://media.istockphoto.com/id/1390530914/photo/black-and-white-horse-portrait-in-bridle.webp?b=1&s=170667a&w=0&k=20&c=6-TUirI-R0-J9cCu77zRHTNaHAY5OwM8JPpNqJyKSBw=
Couldn't find element [53, 92]. Checking if Xpath has changed
Timeout! Exiting the loop.
Downloaded element [53, 93]. URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTmcRP8dqQH2bJR1zAjHEftTbQlgCjqQN4kRg&usqp=CAU
Downloaded element [53, 94]. URL: https://aboutmurals.ca/wp-content/uploads/2017/10/Blue-Horse-Wallpaper-Bedroom-About-Murals.jp